<a href="https://colab.research.google.com/github/kangmg/XequiNet/blob/master/notebook/xequinet_ase.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
try: import xequinet
except ModuleNotFoundError:
    !pip install git+https://github.com/kangmg/XequiNet.git

In [2]:
#@title Install conda

try:
  import condacolab
  condacolab.check()
except:
  %pip install condacolab
  import condacolab
  condacolab.install()

✨🍰✨ Everything looks OK!


In [3]:
# xtb calculator used for delta learning model
!conda install tblite
!conda install tblite-python

Channels:
 - conda-forge
Platform: linux-64
Solving environment: ...working... done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - tblite


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2024.8.30  |       hbcca054_0         155 KB  conda-forge
    certifi-2024.8.30          |     pyhd8ed1ab_0         160 KB  conda-forge
    dftd4-3.6.0                |       hf49bc11_0         587 KB  conda-forge
    libblas-3.9.0              |25_linux64_openblas          15 KB  conda-forge
    libgcc-14.2.0              |       h77fa898_1         829 KB  conda-forge
    libgcc-ng-14.2.0           |       h69a702a_1          53 KB  conda-forge
    libgfortran-14.2.0         |       h69a702a_1          53 KB  conda-forge
    libgfortran-ng-14.2.0      |       h69a702a_1          53 KB  conda-forge
    libgfortran5-14.2.0        |       hd5240d6_1

In [6]:
# re-check tblite is installed
try:
  from tblite.ase import TBLite
except:
  print('tblite is not callable.\n')
  !conda install -q tblite
  !conda install -q tblite-python

  print('\nRestart session . . .')

  import os
  os._exit(0)

In [8]:
# convert pt to jit

# direct learning model
!xeqjit -c spice_1109.pt -o md.jit --md
!xeqjit -c spice_1109.pt -o geo.jit

# delta learning model
!xeqjit -c d-spice_1111.pt -o d_md.jit --md
!xeqjit -c d-spice_1111.pt -o d_geo.jit

/usr/local/lib/python3.10/dist-packages/xequinet/interface/__init__.py:17: UserWarning: i-PI is not installed, i-PI driver will not be performed.
  warnings.warn("i-PI is not installed, i-PI driver will not be performed.")
/usr/local/lib/python3.10/dist-packages/xequinet/interface/__init__.py:17: UserWarning: i-PI is not installed, i-PI driver will not be performed.
  warnings.warn("i-PI is not installed, i-PI driver will not be performed.")
/usr/local/lib/python3.10/dist-packages/xequinet/interface/__init__.py:17: UserWarning: i-PI is not installed, i-PI driver will not be performed.
  warnings.warn("i-PI is not installed, i-PI driver will not be performed.")
/usr/local/lib/python3.10/dist-packages/xequinet/interface/__init__.py:17: UserWarning: i-PI is not installed, i-PI driver will not be performed.
  warnings.warn("i-PI is not installed, i-PI driver will not be performed.")


In [9]:
import warnings
warnings.filterwarnings('ignore')

<br>
<br>
<font color=skyblue size=5> direct learned model </font>
<br>
<br>


In [12]:
# md type model ase test

from xequinet.interface import XeqCalculator
from ase.build import molecule

atoms = molecule('C6H6')

calc = XeqCalculator(ckpt_file='md.jit', model_type='md')

atoms.calc = calc

atoms.get_potential_energy()

-6324.118982349497

In [13]:
# geometry type model ase test

from xequinet.interface import XeqCalculator
from ase.build import molecule

atoms = molecule('C6H6')

calc = XeqCalculator(ckpt_file='geo.jit', model_type='geometry')

atoms.calc = calc

atoms.get_potential_energy()

-6324.118975673778

In [16]:
from xequinet.interface import XeqCalculator
from ase.optimize import BFGS
from ase.build import molecule

atoms = molecule('C6H6')

calc = XeqCalculator(ckpt_file='geo.jit', model_type='geometry')

atoms.calc = calc

opt = BFGS(atoms)
opt.run(fmax=0.05)

      Step     Time          Energy          fmax
BFGS:    0 02:54:21    -6324.118976        0.228401
BFGS:    1 02:54:23    -6324.124423        0.185055
BFGS:    2 02:54:23    -6324.131480        0.032614


True

<br>
<br>
<font color=skyblue size=5> $\Delta$-learned model </font>
<br>
<br>


In [14]:
# geo type delta learned model
from ase.calculators.mixing import SumCalculator
from xequinet.interface import XeqCalculator
from tblite.ase import TBLite
from ase.build import molecule

atoms = molecule('C6H6')

delta_calc = SumCalculator([
    XeqCalculator(ckpt_file='d_geo.jit', model_type='geometry'), # geo type
    TBLite(verbosity=0, method='GFN2-xTB'), # xtb
])

atoms.calc = delta_calc

atoms.get_potential_energy()

-6324.108374815414

In [17]:
# geo type delta learned model
from ase.calculators.mixing import SumCalculator
from xequinet.interface import XeqCalculator
from tblite.ase import TBLite
from ase.build import molecule
from ase.optimize import BFGS

atoms = molecule('C6H6')

delta_calc = SumCalculator([
    XeqCalculator(ckpt_file='d_geo.jit', model_type='geometry'), # geo type
    TBLite(verbosity=0, method='GFN2-xTB'), # xtb
])

atoms.calc = delta_calc

opt = BFGS(atoms)
opt.run(fmax=0.05)

      Step     Time          Energy          fmax
BFGS:    0 02:54:58    -6324.108375        0.229776
BFGS:    1 02:55:01    -6324.114082        0.190545
BFGS:    2 02:55:02    -6324.121398        0.040251


True

In [23]:
# md type delta learned model

from ase.calculators.mixing import SumCalculator
from xequinet.interface import XeqCalculator
from tblite.ase import TBLite
from ase.build import molecule

atoms = molecule('C6H6')

delta_calc = SumCalculator([
    XeqCalculator(ckpt_file='d_md.jit', model_type='md'), # md type
    TBLite(verbosity=0, method='GFN2-xTB'), # xtb
])

atoms.calc = delta_calc

atoms.get_potential_energy()

-6324.108375947902